# CS5830 Final Project

## Ensemble Boosting & Bagging

## Imports

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## Preprocessing

## Base Models

Note: LogisticRegression - SVC - Naive Bayes - Decision Tree - KNeighborsClassifier - Neural Net

### High Bias Models

#### Low Scores

#### High Scores

### High Variance Models

#### Low Scores

#### High Scores

## Boosted Ensembles

Note: AdaBoost - XGBoost - GammaBoost

## Bagging Ensembles

Note: sklearn.ensemble.BaggingClassifier